In [ ]:
%cd ..\src

In [ ]:
import torch
from torch.utils.data import DataLoader

from configs.utils import get_config
from echovpr.datasets.image_ds import ImageDataset
from echovpr.datasets.utils import save_np_file, load_np_file
from echovpr.models.netvlad_encoder import NetVLADEncorder

import numpy as np

In [ ]:
config = get_config("configs\\train_mlp_nordland_full.ini", log=False)

In [ ]:
summer_ds = load_np_file(config['dataset_nordland_summer_dataset_file_path'])
winter_ds = load_np_file(config['dataset_nordland_winter_dataset_file_path'])
spring_ds = load_np_file(config['dataset_nordland_spring_dataset_file_path'])
fall_ds = load_np_file(config['dataset_nordland_fall_dataset_file_path'])

In [ ]:
summer_dataset = ImageDataset(summer_ds['image_names'], config['dataset_root_dir'], config)
summer_dataLoader = DataLoader(summer_dataset, batch_size=int(config['dataset_netvlad_processing_batchsize']), num_workers=int(config['dataset_netvlad_processing_threads']), shuffle=False)

winter_dataset = ImageDataset(winter_ds['image_names'], config['dataset_root_dir'], config)
winter_dataLoader = DataLoader(winter_dataset, batch_size=int(config['dataset_netvlad_processing_batchsize']), num_workers=int(config['dataset_netvlad_processing_threads']), shuffle=False)

spring_dataset = ImageDataset(spring_ds['image_names'], config['dataset_root_dir'], config)
spring_dataLoader = DataLoader(spring_dataset, batch_size=int(config['dataset_netvlad_processing_batchsize']), num_workers=int(config['dataset_netvlad_processing_threads']), shuffle=False)

fall_dataset = ImageDataset(fall_ds['image_names'], config['dataset_root_dir'], config)
fall_dataLoader = DataLoader(fall_dataset, batch_size=int(config['dataset_netvlad_processing_batchsize']), num_workers=int(config['dataset_netvlad_processing_threads']), shuffle=False)

In [ ]:
encoder = NetVLADEncorder(config).eval().cuda()

In [ ]:
def process(model, dataLoader):
    x_processed_list = []

    for x in dataLoader:
        x = x.cuda()
        x_processed = model(x)

        x_processed_list.append(x_processed.cpu())

    return torch.vstack(x_processed_list)

In [ ]:
nordland_summer_repr = process(encoder, summer_dataLoader).numpy()
save_np_file(nordland_summer_repr, config['dataset_nordland_summer_netvlad_repr_file_path'])

nordland_winter_repr = process(encoder, winter_dataLoader).numpy()
save_np_file(nordland_winter_repr, config['dataset_nordland_winter_netvlad_repr_file_path'])

nordland_summer_repr = process(encoder, spring_dataLoader).numpy()
save_np_file(nordland_summer_repr, config['dataset_nordland_spring_netvlad_repr_file_path'])

nordland_winter_repr = process(encoder, fall_dataLoader).numpy()
save_np_file(nordland_winter_repr, config['dataset_nordland_fall_netvlad_repr_file_path'])